# Fermionic Dimer

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

## Model

### Non-Hermitian Hamiltonian:

$$
\begin{align}\begin{split}
    \hat{H} &= \varepsilon_1 c^{\dagger}_1 c^\phantom{\dagger}_1  + \varepsilon_2 c^{\dagger}_2 c^\phantom{\dagger}_2 + J \left(c^{\dagger}_1 c^\phantom{\dagger}_2 + c^{\dagger}_2 c^\phantom{\dagger}_1\right)
\end{split}\end{align}
$$


### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= i \partial_T
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right]    \\
    0 &= i \partial_T
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
\end{split}\end{align}

## Solving

In [ ]:
# final time
T = 5.0

# Hamiltonian
ε₁ = 1.0
ε₂ = -1.0
J = 20.0
H = ComplexF64[ε₁ J; J ε₂];

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

# initial condition
N_0 = 1.0

GL[1, 1, 1, 1] = 1.0im * N_0
GG[:, :, 1, 1] = -1.0im .* diagm([1.0, 1.0]) .+ GL[:, :, 1, 1]

function fv!(out, times, t1, t2)
    out[1] = -1.0im * H * GL[t1, t2]
    out[2] = -1.0im * H * GG[t1, t2]
end

function fd!(out, times, t1, t2)
  fv!(out, times, t1, t2)
  out[1] .-= adjoint(out[1])
  out[2] .-= adjoint(out[2])
end

# Analytic result
ana(t1, t2) = exp(-1.0im * H * t1) * GL[:, :, 1, 1] * exp(1.0im * H * t2)

In [ ]:
sol = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); atol=1e-8, rtol=1e-6);

## Example plots

In [ ]:
idx1 = 1
idx2 = 1;

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(J .* sol.t, [imag(GL.data[idx1, idx2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="C0")
ax.set_xlim(0, J * 1)
ax.set_ylim(0, N_0)
ax.set_xticks(J .* [0, 0.5, 1])
ax.set_yticks([0, 0.5, 1])
ax.set_xlabel(L"J t")
ax.set_ylabel(L"\operatorname{Im}G^<_{11}(t, t)")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_axisbelow(false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
plot(J .* sol.t, [imag(GL.data[idx1, idx2, k, k] - ana(sol.t[k], sol.t[k])[idx1, idx2]) for k in eachindex(sol.t)], marker="", ms=3.0, ls="-", c="r")
ax.set_xlim(0, J * 5)
ax.set_ylim((-5, 5) .* 1e-3)
ax.set_xticks(J .* [0, 1, 2, 3, 4, 5])
ax.set_xlabel(L"J t")
ax.set_ylabel(L"\operatorname{Im}\left[G^<_{11}(t, t) - \mathcal{G}^<_{11}(t, t)\right]", labelpad=16)
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.yaxis.set_label_position("right")
ax.set_axisbelow(false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
savefig("fermion_example_1.pdf")

## Error scaling

In [ ]:
using LsqFit

In [ ]:
epsilons = [10^(-k) for k in range(3, 10; length=34)]

err_data = []
p_norm = 1

for (k, eps) in enumerate(epsilons)
    print("$k, ")
    
    sol = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); dtini=1e-10, atol=1e-2eps, rtol=eps, kmax=9);
    
    err = norm(
    [GL.data[idx1, idx2, t1, t2] - ana(sol.t[t1], sol.t[t2])[idx1, idx2] for t1 in eachindex(sol.t), t2 in eachindex(sol.t)], p_norm)
    
    push!(err_data, (length(sol.t), err / length(sol.t)^2, eps))
end

In [ ]:
xdata = log10.([x[1] for x in err_data])
ydata = log10.([x[2] for x in err_data]);

In [ ]:
fit_func = (n, p) -> -p[1] .* n .+ p[2];
fit_result = curve_fit(fit_func, xdata, ydata, [2.0, 1]);
coef(fit_result)

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(xdata, map(x -> fit_func(x, coef(fit_result)), xdata), "--k", lw=2,
     label=L"\mathcal{O}(h^{%$(floor(coef(fit_result)[1], sigdigits=4))})")
plot(xdata, ydata, "o", lw=0, ms=6,
     markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="#2D5FAA")
ax.set_xlabel(L"\log(n)")
ax.set_ylabel(L"\log(\epsilon_{\mathrm{abs}})")
legend(loc="best", handlelength=1.8, frameon=false, borderpad=0, labelspacing=0)

ax = subplot(122)
plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "o", ms=5,
     markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="#2D5FAA")
ax.set_xlim(-2, -10.3)
ax.set_xticks([-2, -4, -6, -8, -10])
ax.yaxis.set_label_position("right")
ax.set_xlabel(L"\log(\texttt{rtol})")
ax.set_ylabel(L"\log(n)", labelpad=16)

tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
# savefig("error_scaling_fermion.pdf")